### Evaluating predicted 'artifact-free' predicted images by the trained model.

In [ ]:
import os
import sys
import csv
import math
import scipy.io
import random
import copy
import time

import numpy as np
import matplotlib.pyplot as plt
import imshow_grid as ig
from datetime import datetime
from skimage.metrics import structural_similarity as ssim
from IPython import display
from IPython.display import clear_output
from dataclasses import dataclass

import ImportantFunctions as ImFunc

#### 1. Setup

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="?"

# check GPU in-use:
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())
print(tf.test.is_gpu_available())

In [ ]:
# set paths:
checkpoint_dir, TB_logs_dir = ???, ???
test_path = ???
csv_path = result_path = ???, ??? #for storing evaluation results and predicted images
PatLungDict = ??? #a dictionary to select only CT slices corresponding to lung region 

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

if not os.path.exists(TB_logs_dir):
    os.makedirs(TB_logs_dir)
    
pat_gt_path, pat_input_path, pat_pred_path = ???,???, ???
pat_gt_path_img, pat_pred_path_img = ???, ???
                 
if not os.path.exists(pat_gt_path):
    os.makedirs(pat_gt_path_res)
if not os.path.exists(pat_input_path):
    os.makedirs(pat_input_path_res)
if not os.path.exists(pat_pred_path):
    os.makedirs(pat_pred_path_res)
if not os.path.exists(pat_gt_path_img):
    os.makedirs(pat_gt_path_img)
if not os.path.exists(pat_pred_path_img):
    os.makedirs(pat_pred_path_img)

In [ ]:
# define dataclasses
@dataclass
class standard_values:
    net_type: str
    epochs: int
    batch_size: int
    learning_rate: float
    geometry: str
    checkpoint_dir:str=checkpoint_dir
    val_path:str=val_path
    train_path:str=train_path
    test_path=test_path
    log_dir:str=TB_logs_dir
   
@dataclass
class specific_run:
    angle_list: list
    chosen_date: str
    wanted_epoch: int

In [ ]:
# get the epoch and model of interest
netsdict_res_short={
    ("unet", "general"):standard_values(net_type="Unet", epochs=30, batch_size=6, learning_rate=0.001, geometry="parallel"),
    ("unet", 16):specific_run(angle_list=[16], 
                              chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("unet", 32):specific_run(angle_list=[32], 
                              chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("unet", 64):specific_run(angle_list=[64], 
                              chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("unet", 128):specific_run(angle_list=[128], 
                               chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("unet", 256):specific_run(angle_list=[256], 
                               chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("unet", 512):specific_run(angle_list=[512], 
                               chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", "general"):standard_values(net_type="dualUnet", epochs=30, batch_size=6, learning_rate=0.001, geometry="parallel"),
    ("dualunet", 16):specific_run(angle_list=[16], 
                                  chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", 32):specific_run(angle_list=[32], 
                                  chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", 64):specific_run(angle_list=[64], 
                                  chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", 128):specific_run(angle_list=[128], 
                                   chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", 256):specific_run(angle_list=[256], 
                                   chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??),
    ("dualunet", 512):specific_run(angle_list=[512], 
                                   chosen_date="YYYY_MM_DD__HH_MM", wanted_epoch=??)
    }

#### 2. Evaluate and save results

In [ ]:
training_angles=[[16], [32], [64], [128], [256], [512]]
testing_angles=[[16], [32], [64], [128], [256], [512]]
nets=["unet", "dualunet"]
header=["net_type", "training_angle", "testing_angle", "metric", "value"] #for csv file

In [ ]:
# open the file in the write mode
f = open('evalResults.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)
for net in nets:
    for training_angle in training_angles:
        for testing_angle in testing_angles:
            if (len(training_angle)==1 and training_angle==testing_angle): 
                print("train: %s, test:%s"%(str(training_angle), str(testing_angle)))
                
                imgsres, valsres, imgsimg, valsimg = ImFunc.get_evaluation_residual(netsdict_res_short[net, "general"],netsdict_res_short[net, int(ImFunc.get_anglenames(copy.deepcopy(training_angle)))], return_images=True, diff_testdata=[True, testing_angle], LungDict=PatLungDict)

                writer.writerow([net, training_angle, testing_angle, "psnr_res", np.mean(valsres[0])])
                writer.writerow([net, training_angle, testing_angle, "mse_res", np.mean(valsres[1])])
                writer.writerow([net, training_angle, testing_angle, "ssim_res", np.mean(valsres[2])])
                writer.writerow([net, training_angle, testing_angle, "psnr_img", np.mean(valsimg[0])])
                writer.writerow([net, training_angle, testing_angle, "mse_img", np.mean(valsimg[1])])
                writer.writerow([net, training_angle, testing_angle, "ssim_img", np.mean(valsimg[2])])
                
                #iterate over patients
                for i in range(len(imgsres[0])):
                    #iterate over slices
                    for x in range(len(imgsres[0][i])):
                        pat_name=imgsres[-1][i][x].split("_")[0]
            
                        np.save(pat_gt_path+"/"+imgsres[-1][i][x], imgsres[2][i][x])
                        np.save(pat_input_path+"/"+imgsres[-1][i][x], imgsres[1][i][x])
                        np.save(pat_pred_path+"/"+imgsres[-1][i][x], imgsres[0][i][x])
                        
                        np.save(pat_gt_path_img+"/"+imgsimg[-1][i][x], imgsimg[2][i][x])
                        np.save(pat_pred_path_img+"/"+imgsimg[-1][i][x], imgsimg[0][i][x])
                        
            # close the file
f.close()